**Nadia Serafin**

__Rozwiązanie pracy domowej nr 5__

## 1. Przygotowanie danych

1. Wykonaj `import` potrzebnych pakietów (oprócz poniżej wymienionych), tak aby umieszczony w tym notebooku kod działał.

In [1]:
import time
import sqlite3
import pickle
import pandas as pd

2. Wczytaj ramki danych, na których będziesz dalej pracował

**UWAGA:**
* Pliki muszą znajdować się w katalogu "travel_stackexchange_com" w tym samym katalogu co ten notebook (plik .ipynb).
* Nazwy tabel muszą być *zgodne z instrukcją zamieszczoną w treści pracy domowej*.


In [2]:
# DOPISZ TU ODPOWIEDNI KOD
Comments = pd.read_csv("travel_stackexchange_com/Comments.csv")
Posts = pd.read_csv("travel_stackexchange_com/Posts.csv")
Users = pd.read_csv("travel_stackexchange_com/Users.csv")

3. Przygotuj bazę danych wykonując poniższą komórkę.

In [3]:
# NIE MODYFIKUJ TEJ KOMÓRKI
SCIEZKA_BAZY = './pd5_baza.db'  # Ścieżka do pliku z bazą danych ('./' oznacza bieżący katalog, czyli będzie to plik w tym samym katalogu, co ten notebook).
with sqlite3.connect(SCIEZKA_BAZY) as conn: # połączenie do bazy danych
    # wewnątrz bloku `with` mamy dostępny obiekt połączenia, które jest automatycznie zamykane po jego opuszczeniu.
    Comments.to_sql("Comments", conn, if_exists='replace')  # jeżeli ramka danych już istnieje, to jest nadpisywana.
    Posts.to_sql("Posts", conn, if_exists='replace')
    Users.to_sql("Users", conn, if_exists='replace')

## 2. Wyniki zapytań SQL

Wykonaj zapytania sql. Poniższy kod zapisze też wynik do pliku bazy - potem można go z niej odczytać.

**Uwaga**: Zapytania powinny się wykonywać nie dłużej niż kilka sekund każde,
jednak czasem występują problemy zależne od systemu, np. pod Linuxem zapytania 3 i 5 potrafią zająć
odp. kilka minut i ponad godzinę. Żeby obejść ten problem pod koniec tej sekcji pokazane jest, jak
zapisać wyniki zapytań do tymczasowych plików `pickle`.


In [4]:
# NIE MODYFIKUJ TEJ KOMÓRKI (poza ew. zakomentowaniem wywoływania zapytań)
zapytanie_1 = """
SELECT Location, SUM(UpVotes) as TotalUpVotes
FROM Users
WHERE Location != ''
GROUP BY Location
ORDER BY TotalUpVotes DESC
LIMIT 10
"""

zapytanie_2 = """
SELECT STRFTIME('%Y', CreationDate) AS Year,
       STRFTIME('%m', CreationDate) AS Month,
       COUNT(*) AS PostsNumber, MAX(Score) AS MaxScore
FROM Posts
WHERE PostTypeId IN (1, 2)
GROUP BY Year, Month
HAVING PostsNumber > 1000
"""

zapytanie_3 = """
SELECT Id, DisplayName, TotalViews
FROM (
        SELECT OwnerUserId, SUM(ViewCount) as TotalViews
        FROM Posts
        WHERE PostTypeId = 1
        GROUP BY OwnerUserId
    ) AS Questions
JOIN Users
ON Users.Id = Questions.OwnerUserId
ORDER BY TotalViews DESC
LIMIT 10
"""

zapytanie_4 = """
SELECT DisplayName, QuestionsNumber, AnswersNumber, Location, Reputation, UpVotes, DownVotes
FROM (
        SELECT *
        FROM (
                SELECT COUNT(*) as AnswersNumber, OwnerUserId
                FROM Posts
                WHERE PostTypeId = 2
                GROUP BY OwnerUserId
            ) AS Answers
        JOIN
            (
                SELECT COUNT(*) as QuestionsNumber, OwnerUserId
                FROM Posts
                WHERE PostTypeId = 1
                GROUP BY OwnerUserId
            ) AS Questions
        ON Answers.OwnerUserId = Questions.OwnerUserId
        WHERE AnswersNumber > QuestionsNumber
        ORDER BY AnswersNumber DESC
        LIMIT 5
    ) AS PostsCounts
JOIN Users
ON PostsCounts.OwnerUserId = Users.Id
"""

zapytanie_5 = """
SELECT Title, CommentCount, ViewCount, CommentsTotalScore, DisplayName, Reputation, Location
FROM (
        SELECT Posts.OwnerUserId, Posts.Title, Posts.CommentCount, Posts.ViewCount,
               CmtTotScr.CommentsTotalScore
        FROM (
                SELECT PostId, SUM(Score) AS CommentsTotalScore
                FROM Comments
                GROUP BY PostId
            ) AS CmtTotScr
        JOIN Posts ON Posts.Id = CmtTotScr.PostId
        WHERE Posts.PostTypeId=1
    ) AS PostsBestComments
JOIN Users ON PostsBestComments.OwnerUserId = Users.Id
ORDER BY CommentsTotalScore DESC
LIMIT 10
"""

def wykonaj_zapytanie(zap, k, conn_):
    """
    Wykonuje zapytanie zap (str) o numerze k (int) przy użyciu połączenia z bazą w obiekcie conn_.
    Wypisuje czas wykonania zapytania w sekundach (dokładnie tzw. Wall Clock Time, czyli czas rzeczywisty, który upłynął).
    Zwraca wynik zapytania jako ramkę danych Pandas.
    """
    t0 = time.time()  # zapisanie obecnego czasu do zmiennej
    wynik = pd.read_sql_query(zap, conn_)  # wykonanie zapytania
    t = time.time() - t0  # czas (w sekundach), jaki zajęło wykonania zapytania
    print(f'Wykonano zapytanie {k} w {t:.2f}s.')
    return wynik

# Poniższy blok with wykonuje wszystkie 5 zapytań;
# Wyniki umieszcza w zmiennych sql_i.
with sqlite3.connect(SCIEZKA_BAZY) as conn:
#     sql_1 = wykonaj_zapytanie(zapytanie_1, 1, conn)
    sql_2 = wykonaj_zapytanie(zapytanie_2, 2, conn)
#     sql_3 = wykonaj_zapytanie(zapytanie_3, 3, conn)
#     sql_4 = wykonaj_zapytanie(zapytanie_4, 4, conn)
#     sql_5 = wykonaj_zapytanie(zapytanie_5, 5, conn)

Wykonano zapytanie 2 w 0.18s.


Moduł `pickle` to system serializacji (konwersji danych do postaci mogącej być zapisaną na dysku/przesłaną przez sieć) obiektów Pythona.
W przeciwieństwie do plików np `csv` można zapisywać niemal dowonly obiekt Pythona. Minusem jest niemożliwość przeczytania takiego obiektu inaczej, niż przez program napisany w Pythonie.

Poniższy kod zapisze wyniki zapytań (ramiki danych Pandas) do takich plików, aby można je potem było szybko odczytać.
**Nie trzeba z niego korzystać, jeżeli nie ma problemów z wykonaniem zapytań SQL.**

In [6]:
# Zapisanie każdej z ramek danych opisujących wyniki zapytań SQL do osobnego pliku pickle.
for i, df in enumerate([sql_1, sql_2, sql_3, sql_4, sql_5], 1):
    df.to_pickle(f'sql_{i}.pkl.gz')

In [ ]:
# Wczytanie policzonych uprzednio wyników z plików pickle (możesz to zrobić, jeżeli zapytania wykonują się za długo).
sql_1, sql_2, sql_3, sql_4, sql_5 = [
    pd.read_pickle(f'sql_{i}.pkl.gz') for i in range(1, 5 + 1)
]

## 3. Wyniki zapytań SQL odtworzone przy użyciu metod pakietu Pandas.

Wynikowa ramka danych do zapytania 1 popwinna nazwyać się `pandas_1`, do drugiego `pandas_2` itd.

### Zadanie 1

In [46]:
try:
    pandas_1 = Users[Users['Location'] != '']\
               .groupby('Location')['UpVotes']\
               .sum()\
               .rename("TotalUpVotes")\
               .sort_values(ascending=False)\
               .head(10)\
               .reset_index()
    
    # sprawdzenie równoważności wyników
    print(pandas_1.equals(sql_1))

except Exception as e:
    print("Zad. 1: niepoprawny wynik.")
    print(e)

True


### Zadanie 2

In [23]:
try:
    pandas_2 = Posts.copy()
    pandas_2["CreationDate"] = pd.to_datetime(pandas_2["CreationDate"])
    pandas_2["Year"] = pandas_2["CreationDate"].dt.strftime("%Y")
    pandas_2["Month"] = pandas_2["CreationDate"].dt.strftime("%m")
    
    pandas_2 = pandas_2[pandas_2["PostTypeId"].isin([1, 2])]
    pandas_2 = pandas_2.groupby(["Year", "Month"]).agg(PostsNumber=("PostTypeId", "size"), MaxScore=("Score", "max")).reset_index()
    pandas_2 = pandas_2[pandas_2["PostsNumber"] > 1000]
    pandas_2 = pandas_2.reset_index(drop = True)

    # sprawdzenie równoważności wyników
    print(pandas_2.equals(sql_2))

except Exception as e:
    print("Zad. 2: niepoprawny wynik.")
    print(e)

True


### Zadanie 3

In [18]:
try:
    temp = Posts[Posts["PostTypeId"] == 1]\
           .groupby("OwnerUserId")["ViewCount"]\
           .sum()\
           .rename("TotalViews")\
           .reset_index()\
           .merge(Users[["Id", "DisplayName"]], left_on = "OwnerUserId", right_on = "Id")\
           .sort_values(by = "TotalViews", ascending = False)\
           .reset_index()\
           .head(10)
    
    pandas_3 = temp[["Id", "DisplayName", "TotalViews"]]
    
    # sprawdzenie równoważności wyników
    print(pandas_3.equals(sql_3))

except Exception as e:
    print("Zad. 3: niepoprawny wynik.")
    print(e)

True


### Zadanie 4

In [29]:
try:
    Answers = Posts[Posts["PostTypeId"] == 2]\
              .groupby("OwnerUserId")\
              .size()\
              .rename("AnswersNumber")\
              .reset_index()
    
    Questions = Posts[Posts["PostTypeId"] == 1]\
                .groupby("OwnerUserId")\
                .size()\
                .rename("QuestionsNumber")\
                .reset_index()
    
    PostsCounts = pd.merge(Answers, Questions, left_on = "OwnerUserId", right_on = "OwnerUserId")
    PostsCounts = PostsCounts[PostsCounts.AnswersNumber > PostsCounts.QuestionsNumber]\
                  .sort_values(by = "AnswersNumber", ascending = False)\
                  .reset_index()\
                  .head(5)
    
    pandas_4 = pd.merge(PostsCounts, Users, left_on = "OwnerUserId", right_on = "Id")
    pandas_4 = pandas_4[["DisplayName", "QuestionsNumber", "AnswersNumber", "Location", "Reputation", "UpVotes", "DownVotes"]]

    # sprawdzenie równoważności wyników
    print(pandas_4.equals(sql_4))

except Exception as e:
    print("Zad. 4: niepoprawny wynik.")
    print(e)

True


### Zadanie 5

In [58]:
try:
    CmtTotScr = Comments\
                .groupby("PostId")["Score"]\
                .sum()\
                .rename("CommentsTotalScore")\
                .reset_index()
    
    PostsBestComments = Posts[Posts["PostTypeId"] == 1]
    PostsBestComments = pd.merge(Posts, CmtTotScr, left_on = "Id", right_on = "PostId")
    PostsBestComments = PostsBestComments[["OwnerUserId", "Title", "CommentCount", "ViewCount", "CommentsTotalScore"]]
    
    pandas_5 = pd.merge(PostsBestComments, Users, left_on = "OwnerUserId", right_on = "Id")\
               .dropna(subset = ["Title"])\
               .sort_values(by = "CommentsTotalScore", ascending = False)\
               .reset_index()\
               .head(10)
    pandas_5 = pandas_5[["Title", "CommentCount", "ViewCount", "CommentsTotalScore", "DisplayName", "Reputation", "Location"]]
    pandas_5

    # sprawdzenie równoważności wyników
    print(pandas_5.equals(sql_5))

except Exception as e:
    print("Zad. 5: niepoprawny wynik.")
    print(e)

True
